In [1]:
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
from sklearn import *
import glob

import gc
from tqdm.auto import tqdm

In [2]:
path = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/'

In [3]:
train_defog = glob.glob(path+'train/defog/**')
train_tdcsfog = glob.glob(path+'train/tdcsfog/**')

In [4]:
test_defog = glob.glob(path+'test/defog/**')
test_tdcsfog = glob.glob(path+'test/tdcsfog/**')

In [5]:
def read_file(file_path):
    df = pd.read_csv(file_path)
    df['Id'] = file_path.split('/')[-1].split('.')[0]
    df['Module'] = file_path.split('/')[-2]
    return df

In [6]:
train_defog = pd.concat([read_file(file_path) for file_path in tqdm(train_defog)])
train_tdcsfog = pd.concat([read_file(file_path) for file_path in tqdm(train_tdcsfog)])

  0%|          | 0/91 [00:00<?, ?it/s]

  0%|          | 0/833 [00:00<?, ?it/s]

In [7]:
subjects = pd.read_csv(path+'subjects.csv')
tasks = pd.read_csv(path+'tasks.csv')

In [8]:
# not needed
subjects.drop(['Visit'], axis=1, inplace=True)
subjects.head()

,Subject,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ
0,04fcdb,63,M,3.0,30.0,NaN,0
1,05595e,56,M,8.0,28.0,NaN,0
2,0967b2,59,M,10.0,38.0,48.0,19
3,0967b2,59,M,10.0,37.0,44.0,13
4,097078,70,F,10.0,27.0,50.0,20


In [9]:
train_df = pd.concat([train_defog, train_tdcsfog])

In [10]:
train_df.head()

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Valid,Task,Id,Module
0,0,-1.002697,0.022371,0.068304,0,0,0,False,False,be9d33541d,defog
1,1,-1.002641,0.019173,0.066162,0,0,0,False,False,be9d33541d,defog
2,2,-0.999820,0.019142,0.067536,0,0,0,False,False,be9d33541d,defog
3,3,-0.998023,0.018378,0.068409,0,0,0,False,False,be9d33541d,defog
4,4,-0.998359,0.016726,0.066448,0,0,0,False,False,be9d33541d,defog


In [11]:
train_df.fillna(0,inplace=True)

In [12]:
features=['Time', 'AccV', 'AccML', 'AccAP']
Targets=['StartHesitation', 'Turn' , 'Walking']

In [13]:
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(train_df[features], train_df[Targets], test_size=.20, random_state=42)

In [14]:
del train_df
gc.collect()

126

In [15]:
regression_model = ensemble.RandomForestRegressor(n_estimators=50, max_depth=6, n_jobs=-1, random_state=42)
regression_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=6, n_estimators=50, n_jobs=-1, random_state=42)

In [16]:
submission_sample = pd.read_csv(path+'sample_submission.csv')
test = glob.glob(path+'test/**/**')

In [17]:
submission_sample['t'] = 0
submission = []

for test_file in test:
    df = pd.read_csv(test_file)
    df['Id'] = test_file.split('/')[-1].split('.')[0]
    df = df.fillna(0).reset_index(drop=True)
    res = pd.DataFrame(np.round(regression_model.predict(df[features]),3), columns=['StartHesitation', 'Turn' , 'Walking'])
    df = pd.concat([df,res], axis=1)
    df['Id'] = df['Id'].astype(str) + '_' + df['Time'].astype(str)
    submission.append(df[['Id','StartHesitation', 'Turn' , 'Walking']])

In [18]:
submission = pd.concat(submission)
submission = pd.merge(submission_sample[['Id','t']], submission, how='left', on='Id').fillna(0.0)
submission[['Id','StartHesitation', 'Turn' , 'Walking']].to_csv('submission.csv', index=False)